<a href="https://colab.research.google.com/github/skynunu/2021_cau_oss_hackathon/blob/main/hackathon2_team20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.utils import np_utils
import numpy as np

# 데이터셋 로드 (Training dataset: CIFAR10, test dataset: CIFAR10 & CIFAR10의 변형)
(x_train, y_train), (x_test1, y_test1) = keras.datasets.cifar10.load_data()

test_ds2 = tfds.load('cifar10_1/v6', split='test', shuffle_files=False, batch_size=-1)
test_ds2 = tfds.as_numpy(test_ds2)
x_test2, y_test2 = test_ds2['image'], test_ds2['label']

#분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test1 = np_utils.to_categorical(y_test1)
y_test2 = np_utils.to_categorical(y_test2)

# 총 클래스 개수
num_classes = y_train.shape[1]
input_shape = x_train.shape[1:]

In [8]:
# 데이터 전처리 (예: normalization)
# 원본 데이터와 전처리 후 데이터를 구분하기 위해, 변수명 x_train_after, x_test1_after, x_test2_after를 변경하지 말 것
x_train_after = x_train / 255.0
x_test1_after = x_test1 / 255.0
x_test2_after = x_test2 / 255.0

x_train_after= x_train_after.astype('float32')
x_test1_after= x_test1_after.astype('float32')
x_test2_after= x_test2_after.astype('float32')

In [9]:
# train dataset data augmentation
batch_size =64
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True )

train_dataset = datagen.flow(x_train_after, y_train, batch_size=batch_size)

In [10]:
# batch 사이즈 설정 test1_after, test2_after
BATCH_SIZE =64

test1_dataset = tf.data.Dataset.from_tensor_slices((x_test1_after, y_test1))
test2_dataset = tf.data.Dataset.from_tensor_slices((x_test2_after, y_test2))

test1_dataset = test1_dataset.batch(BATCH_SIZE)
test2_dataset = test2_dataset.batch(BATCH_SIZE)


In [11]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D,Activation, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.callbacks import ModelCheckpoint

In [12]:
     model = Sequential([ 
        Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),                 
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        Conv2D(512, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(512, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(512, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        
        #Flatten(),
        GlobalAveragePooling2D(),
        Dropout(0.5),
         
        Dense(512),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        
        Dense(256),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),    

        Dense(10, activation='softmax'),

        
     ])
 

In [13]:
# model compile
from keras import optimizers
#'adam'
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# model 체크포인트 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')

In [17]:
# model early point 설정
from keras.callbacks import EarlyStopping
earlystopping = EarlyStopping
earlystopping = EarlyStopping(monitor='val_loss',  
                              patience=12,         
                             )
# 모니터 기준 설정 (val loss) 
# 12회 Epoch동안 개선되지 않는다면 종료 (patience)

In [ ]:
#model fit
model.fit(train_dataset, epochs = 200,  callbacks=[cp_callback, earlystopping], validation_data=(test2_dataset))

Epoch 1/200
782/782 [==============================] - 58s 74ms/step - loss: 1.7363 - accuracy: 0.3739 - val_loss: 1.8186 - val_accuracy: 0.3950

Epoch 00001: val_accuracy improved from -inf to 0.39500, saving model to /content/checkpoint_entire_best.h5
Epoch 2/200
782/782 [==============================] - 57s 72ms/step - loss: 1.1533 - accuracy: 0.5988 - val_loss: 1.5041 - val_accuracy: 0.4945

Epoch 00002: val_accuracy improved from 0.39500 to 0.49450, saving model to /content/checkpoint_entire_best.h5
Epoch 3/200
782/782 [==============================] - 57s 73ms/step - loss: 0.9258 - accuracy: 0.6921 - val_loss: 1.5508 - val_accuracy: 0.5300

Epoch 00003: val_accuracy improved from 0.49450 to 0.53000, saving model to /content/checkpoint_entire_best.h5
Epoch 4/200
782/782 [==============================] - 57s 73ms/step - loss: 0.8042 - accuracy: 0.7359 - val_loss: 1.5026 - val_accuracy: 0.5310

Epoch 00004: val_accuracy improved from 0.53000 to 0.53100, saving model to /content/c

In [ ]:
save_path = '/content/'
team_name = 'team20_transferlearning'

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_'+ team_name + '.h5')

### 모델 평가


In [ ]:
save_path = '/content/'
team_name = 'team20_transferlearning'

model = keras.models.load_model(save_path + 'model_entire_' + team_name + '.h5')

model.evaluate(x_test1_after, y_test1)
model.evaluate(x_test2_after, y_test2)